In [1]:
import pandas as pd

In [14]:
df = pd.read_csv('Cleaned.csv', parse_dates=['datetime'])

In [15]:
df.head()

,datetime,Date,hour,min,latitude,longitude,speed,truck,svr1,svr2,svr3,svr4,Transfer size,Bitrate,Retransmissions,CWnd,Transfer size-RX,Bitrate-RX,send_data,square_id
0,2022-07-04 05:43:37+10:00,4.0,5.0,43.0,-37.737985,144.849691,76.811700,garbo02,25.5,28.9,27.8,44.5,0.864258,7.25,0.0,0.007080,0.988281,8.29,0.000000,square_94489280583
1,2022-07-04 05:43:38+10:00,4.0,5.0,43.0,-37.738127,144.849794,75.519004,garbo02,35.0,32.0,31.3,42.1,1.300000,10.90,3.0,0.010645,0.972656,8.16,0.000000,square_94489280583
2,2022-07-04 05:43:39+10:00,4.0,5.0,43.0,-37.738249,144.849904,72.846568,garbo02,34.2,30.5,29.8,46.1,1.300000,10.90,0.0,0.010645,0.957031,8.03,0.000000,square_94489280583
3,2022-07-04 05:43:40+10:00,4.0,5.0,43.0,-37.738362,144.850020,70.279696,garbo02,25.0,23.2,21.6,38.7,0.864258,7.25,0.0,0.007080,0.953125,7.99,0.014404,square_94489280583
4,2022-07-04 05:43:41+10:00,4.0,5.0,43.0,-37.738491,144.850142,70.153760,garbo02,20.2,18.7,23.4,34.2,0.926758,7.77,6.0,0.007588,0.927734,7.78,0.000000,square_94489280583


In [20]:
df.set_index('datetime', inplace=True)

In [21]:
import pyproj
proj = pyproj.Proj("EPSG:3111")  # GDA94 / VICGRID
df['x'], df['y'] = proj(df.longitude.values, df.latitude.values)


In [23]:
# Make sure data is sorted by square_id and time
df = df.sort_values(['square_id', 'datetime'])

# Move time into the index
rolled = (
    df                     # time must be the index for time-offset rolling
    .groupby('square_id')['lat_mean']       # group by your zone
    .rolling('5min')                        # 5-minute window
    .mean()                                 # compute the mean of lat_mean in each window
)

# rolled is a Series with a MultiIndex (square_id, time), so align it back on the original df
df['lat_roll_5min'] = (
    rolled
    .reset_index(level=0, drop=True)        # drop the square_id level, keep the time index
    .reindex(df.index)                      # reindex to align with the original df rows
)


In [24]:
df

,Date,hour,min,latitude,longitude,speed,truck,svr1,svr2,svr3,...,square_id,x,y,lat_mean,lat_std,br_ratio,lat_mean_lag_1,lat_mean_lag_3,lat_mean_lag_6,lat_roll_5min
datetime,,,,,,,,,,,,,,,,,,,,,
2022-07-06 09:04:53+10:00,6.0,9.0,4.0,-37.724588,144.750419,38.356772,garbo10,29.8,27.4,25.90,...,square_103079215126,2.477999e+06,2.419563e+06,29.8250,4.543402,1.711538e+00,65.0250,58.5750,67.6000,29.825000
2022-07-06 09:04:54+10:00,6.0,9.0,4.0,-37.724686,144.750371,40.347672,garbo10,43.9,41.8,39.40,...,square_103079215126,2.477994e+06,2.419552e+06,44.9250,6.706899,1.519231e+00,29.8250,59.7750,61.4750,37.375000
2022-07-06 09:04:55+10:00,6.0,9.0,4.0,-37.724789,144.750326,41.845940,garbo10,23.7,26.5,26.40,...,square_103079215126,2.477990e+06,2.419541e+06,30.1000,9.224966,1.451923e+00,44.9250,65.0250,92.0750,34.950000
2022-07-06 09:04:57+10:00,6.0,9.0,4.0,-37.725012,144.750230,45.620316,garbo10,44.7,41.9,40.70,...,square_103079215126,2.477982e+06,2.419516e+06,46.1250,7.571162,1.160000e+07,30.1000,29.8250,58.5750,37.743750
2022-07-06 09:04:58+10:00,6.0,9.0,4.0,-37.725136,144.750193,47.611216,garbo10,22.7,26.2,25.10,...,square_103079215126,2.477979e+06,2.419502e+06,29.0250,8.838316,5.652174e-01,46.1250,44.9250,59.7750,36.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-22 13:28:52+10:00,22.0,13.0,28.0,-37.732543,144.850684,72.766932,garbo02,15.9,15.1,8.97,...,square_94489280583,2.486839e+06,2.418699e+06,16.9675,7.918474,0.000000e+00,22.4250,35.5750,18.6525,28.688160
2022-07-22 13:28:53+10:00,22.0,13.0,28.0,-37.732360,144.850719,73.765160,garbo02,19.0,16.8,9.01,...,square_94489280583,2.486842e+06,2.418719e+06,18.0025,7.481089,0.000000e+00,16.9675,37.4000,36.4000,28.628464
2022-07-22 13:28:54+10:00,22.0,13.0,28.0,-37.732175,144.850758,74.585596,garbo02,26.7,24.7,18.40,...,square_94489280583,2.486845e+06,2.418740e+06,25.9000,6.344026,0.000000e+00,18.0025,22.4250,20.5500,28.613306


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1707962 entries, 2022-07-06 09:04:53+10:00 to 2022-07-22 13:28:56+10:00
Data columns (total 28 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Date              float64
 1   hour              float64
 2   min               float64
 3   latitude          float64
 4   longitude         float64
 5   speed             float64
 6   truck             object 
 7   svr1              float64
 8   svr2              float64
 9   svr3              float64
 10  svr4              float64
 11  Transfer size     float64
 12  Bitrate           float64
 13  Retransmissions   float64
 14  CWnd              float64
 15  Transfer size-RX  float64
 16  Bitrate-RX        float64
 17  send_data         float64
 18  square_id         object 
 19  x                 float64
 20  y                 float64
 21  lat_mean          float64
 22  lat_std           float64
 23  br_ratio          float64
 24  lat_mean_lag_1    float64
 25  

In [26]:
df.to_csv('Cleaned_rolled.csv')